In [16]:
import pandas as pd

# Initialize an empty list to store the precipitation data
total_precipitation = []

# Initialize a list for the date columns (Year, Month, Day)
dates = []

# Loop through each file (from 1.csv to 20.csv)
for i in range(1, 21):
    # Read the CSV file
    filename = f'{i}.csv'
    df = pd.read_csv(filename)
    
    # Extract the 'precipitation' column (assuming it's the 5th column, indexed as 4)
    # If your precipitation column is named, you can use df['precipitation'] instead
    precipitation = df.iloc[:, 4]  # Assuming 'precipitation' is the 5th column
    
    # Add the date columns (Year, Month, Day) to the 'dates' list for later use
    #if len(dates) == 0:
        #dates = df[['YEAR', 'MONTH', 'DAY']]  # Capture the dates only once
    
    # Add the precipitation data to the total_precipitation list
    if len(total_precipitation) == 0:
        total_precipitation = precipitation
    else:
        total_precipitation += precipitation

# Now, total_precipitation holds the combined sum of precipitation across all stations
# Create a new DataFrame with the combined precipitation data and corresponding dates
total_precipitation_df = pd.DataFrame({
    #'Year': dates['YEAR'],
    #'Month': dates['MONTH'],
    #'Day': dates['DAY'],
    'Total Precipitation': total_precipitation/20
})

# Display the combined data
total_precipitation_df.head()
pd.set_option('display.max_rows', 100)  # Display up to 10 rows
total_precipitation_df

# Export the DataFrame to a new CSV file
total_precipitation_df.to_csv('combined_precipitation_data.csv', index=False)

,Total Precipitation
0,-9.8900
1,0.2945
2,0.2010
3,0.0000
4,-4.9910
...,...
8405,-14.9315
8406,-19.8260
8407,-24.6455
8408,-14.8225


In [36]:
import pandas as pd
import os

# Initialize an empty list to store the data
all_data = []

# Loop through all CSV files (1.csv to 20.csv)
for i in range(1, 21):
    # Construct the file name
    file_name = f"{i}.csv"
    
    # Read the data without assuming header
    df = pd.read_csv(file_name, header=None)

    # Clean column names by stripping spaces
    df.columns = df.columns.astype(str).str.strip()

    # Assign column names explicitly
    df.columns = ["COOPID", "YEAR", "MONTH", "DAY", "PRECIPITATION"]
    
    # Strip spaces for each column using `apply`
    df = df.apply(lambda col: col.str.strip() if col.dtype == "object" else col)
    
    # Convert YEAR, MONTH, DAY to numeric, coercing any errors (non-numeric values) into NaN
    df["YEAR"] = pd.to_numeric(df["YEAR"], errors="coerce")
    df["MONTH"] = pd.to_numeric(df["MONTH"], errors="coerce")
    df["DAY"] = pd.to_numeric(df["DAY"], errors="coerce")
    df["PRECIPITATION"] = pd.to_numeric(df["PRECIPITATION"], errors="coerce")
    
    # Drop rows with invalid dates (NaN values in YEAR, MONTH, or DAY)
    df = df.dropna(subset=["YEAR", "MONTH", "DAY"])

    # Combine YEAR, MONTH, DAY to form a DATE column, and handle invalid dates
    df['DATE'] = pd.to_datetime(df[['YEAR', 'MONTH', 'DAY']], errors='coerce')
    
    # Drop rows with invalid dates after conversion
    df = df.dropna(subset=['DATE'])
    
    # Group by DATE and calculate the average precipitation for each date
    daily_avg_precip = df.groupby('DATE')['PRECIPITATION'].mean().reset_index()
    
    # Append the result to the all_data list
    all_data.append(daily_avg_precip)

# Concatenate all data
final_data = pd.concat(all_data, ignore_index=True)

# Group again by DATE to ensure the final average for each date across all files
final_data = final_data.groupby('DATE')['PRECIPITATION'].mean().reset_index()

# Export the final data to a new CSV file
final_data.to_csv('average_precipitation_by_date.csv', index=False)

In [35]:
final_data

,DATE,PRECIPITATION
0,2002-01-01,-9.890000
1,2002-01-02,0.294500
2,2002-01-03,0.201000
3,2002-01-04,0.000000
4,2002-01-05,-4.991000
...,...,...
8396,2024-12-27,-14.931500
8397,2024-12-28,-19.826000
8398,2024-12-29,-24.645500
8399,2024-12-30,-14.822499
